In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from math import sqrt
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.regularizers import l2
from helper import series_to_supervised
from model.cnn import cnn_4layer
from model.mlp import mlp_layer

2024-01-20 21:29:52.341272: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
# https://keras.io/examples/timeseries/timeseries_weather_forecasting/#climate-data-timeseries
data = pd.read_csv("../../data/jena_climate_2009_2016_hourly.csv", index_col=0)
data.fillna(0, inplace=True)
data.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 00:00:00,996.528000,-8.304000,265.118000,-9.120000,93.780000,3.260000,3.058000,0.202000,1.910000,3.068000,1309.196000,0.520000,1.002000,174.460000
2009-01-01 01:00:00,996.525000,-8.065000,265.361667,-8.861667,93.933333,3.323333,3.121667,0.201667,1.951667,3.133333,1307.981667,0.316667,0.711667,172.416667
2009-01-01 02:00:00,996.745000,-8.763333,264.645000,-9.610000,93.533333,3.145000,2.940000,0.201667,1.836667,2.950000,1311.816667,0.248333,0.606667,196.816667
2009-01-01 03:00:00,996.986667,-8.896667,264.491667,-9.786667,93.200000,3.111667,2.898333,0.210000,1.811667,2.906667,1312.813333,0.176667,0.606667,157.083333
2009-01-01 04:00:00,997.158333,-9.348333,264.026667,-10.345000,92.383333,3.001667,2.775000,0.231667,1.733333,2.780000,1315.355000,0.290000,0.670000,150.093333


In [4]:
data.columns

Index(['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
       'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
       'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)',
       'wd (deg)'],
      dtype='object')

### Preprocess

In [5]:
values = data.values

# specify the number of lag hours
n_hours = 24*3
n_features = data.shape[-1]
k = 12
split1 = 0.7
split2 = 0.85

# frame as supervised learning
reframed = series_to_supervised(values, n_hours, k)
print("reframed.shape:", reframed.shape)

reframed.shape: (70046, 1176)


In [6]:
# split into train and test sets
reframed_values = reframed.values
n_train_hours = int(len(reframed_values)*split1)
n_valid_hours = int(len(reframed_values)*split2)

train = reframed_values[:n_train_hours, :]
val = reframed_values[n_train_hours:n_valid_hours, :]
test = reframed_values[n_valid_hours:, :]


# split into input and outputs
n_obs = n_hours * n_features
feature_idx = 5
train_X, train_y = train[:, :n_obs], train[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
val_X, val_y = val[:, :n_obs], val[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
test_X, test_y = test[:, :n_obs], test[:, [n_obs + feature_idx + n_features * i for i in range(k)]]


print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (49032, 1008) (49032, 12) (10507, 1008) (10507, 12) (10507, 1008) (10507, 12)


In [7]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)

val_X = scaler.fit_transform(val_X)
val_y = scaler.fit_transform(val_y)

test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
val_X = val_X.reshape((val_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (49032, 72, 14) (49032, 12) (10507, 72, 14) (10507, 12) (10507, 72, 14) (10507, 12)


### PM threshold

In [8]:
train_X_pm = train_X[:, 0, feature_idx]
print(train_X_pm.shape)

val_X_pm = val_X[:, 0, feature_idx]
print(val_X_pm.shape)

test_X_pm = test_X[:, 0, feature_idx]
print(test_X_pm.shape)

(49032,)
(10507,)
(10507,)


In [9]:
percentile = 95

merged_array = np.concatenate((train_X_pm, val_X_pm, test_X_pm))

percentile_pm = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_pm)

95th Percentile of Daily Rain: 0.48441203148316114


### train_X_filter

In [10]:
train_X_extreme = train_X[train_X_pm > percentile_pm]
print(train_X_extreme.shape)

train_y_extreme = train_y[train_X_pm > percentile_pm]
print(train_y_extreme.shape)

(2206, 72, 14)
(2206, 12)


In [11]:
train_X_normal = train_X[train_X_pm <= percentile_pm]
print(train_X_normal.shape)

train_y_normal = train_y[train_X_pm <= percentile_pm]
print(train_y_normal.shape)

(46826, 72, 14)
(46826, 12)


### val_X_filter

In [12]:
val_X_extreme = val_X[val_X_pm > percentile_pm]
print(val_X_extreme.shape)

val_y_extreme = val_y[val_X_pm > percentile_pm]
print(val_y_extreme.shape)

(486, 72, 14)
(486, 12)


In [13]:
val_X_normal = val_X[val_X_pm <= percentile_pm]
print(val_X_normal.shape)

val_y_normal = val_y[val_X_pm <= percentile_pm]
print(val_y_normal.shape)

(10021, 72, 14)
(10021, 12)


### test_X_filter

In [14]:
test_X_extreme = test_X[test_X_pm > percentile_pm]
print(test_X_extreme.shape)

test_y_extreme = test_y[test_X_pm > percentile_pm]
print(test_y_extreme.shape)

(811, 72, 14)
(811, 12)


In [15]:
test_X_normal = test_X[test_X_pm <= percentile_pm]
print(test_X_normal.shape)

test_y_normal = test_y[test_X_pm <= percentile_pm]
print(test_y_normal.shape)

(9696, 72, 14)
(9696, 12)


### Model

In [54]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 50
EPOCHS = 500
BATCH = 512
opt_num = k
input_shape = train_X.shape[1:]

In [61]:
# trained_cnn = load_model('../../saved_models/pressure_all.h5')
# trained_cnn = load_model('../../saved_models/pressure_all_weighted_IPF_95.h5')
# trained_cnn = load_model('../../saved_models/pressure_all_weighted_EVT_95.h5') 
trained_cnn = load_model('../../saved_models/pressure_all_weighted_META.h5') 

freeze_layer_index = 17

for i, layer in enumerate(trained_cnn.layers):
    #print(i, layer)
    if i <= freeze_layer_index:
        layer.trainable = False


l2_strength = 1e-6  # 1e-6
for i in range(freeze_layer_index+1, len(trained_cnn.layers)):
    trained_cnn.layers[i].kernel_regularizer = l2(l2_strength)


trained_cnn._name = 'all_95_ft'
# trained_cnn.summary()

In [62]:
trained_cnn.compile(optimizer='adam',
                    loss='mse',
                    metrics=['mae']
                   )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint(
                      '../../saved_models/pressure_all_weighted_META_ft_17.h5', 
#                      '../../saved_models/pressure_all_weighted_EVT_95_ft.h5', 
#                      '../../saved_models/pressure_all_weighted_IPF_95_ft.h5', 
#                      '../../saved_models/pressure_all_95_ft.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True,
                    )


trained_cnn.fit(train_X_extreme, train_y_extreme,
                validation_data=(val_X_extreme, val_y_extreme),
                epochs=EPOCHS,
                batch_size=BATCH,
                verbose=2,
                shuffle=True,
                callbacks=[es, mc]
               )

Epoch 1/500
5/5 - 1s - loss: 0.0034 - mae: 0.0426 - val_loss: 0.0050 - val_mae: 0.0551

Epoch 00001: val_mae improved from inf to 0.05507, saving model to ../../saved_models/pressure_all_weighted_META_ft_17.h5
Epoch 2/500
5/5 - 0s - loss: 0.0031 - mae: 0.0396 - val_loss: 0.0040 - val_mae: 0.0462

Epoch 00002: val_mae improved from 0.05507 to 0.04625, saving model to ../../saved_models/pressure_all_weighted_META_ft_17.h5
Epoch 3/500
5/5 - 0s - loss: 0.0029 - mae: 0.0379 - val_loss: 0.0043 - val_mae: 0.0498

Epoch 00003: val_mae did not improve from 0.04625
Epoch 4/500
5/5 - 0s - loss: 0.0028 - mae: 0.0369 - val_loss: 0.0039 - val_mae: 0.0459

Epoch 00004: val_mae improved from 0.04625 to 0.04592, saving model to ../../saved_models/pressure_all_weighted_META_ft_17.h5
Epoch 5/500
5/5 - 0s - loss: 0.0027 - mae: 0.0365 - val_loss: 0.0043 - val_mae: 0.0493

Epoch 00005: val_mae did not improve from 0.04592
Epoch 6/500
5/5 - 0s - loss: 0.0027 - mae: 0.0362 - val_loss: 0.0039 - val_mae: 0.0462


Epoch 00056: val_mae did not improve from 0.04437
Epoch 57/500
5/5 - 0s - loss: 0.0025 - mae: 0.0349 - val_loss: 0.0038 - val_mae: 0.0450

Epoch 00057: val_mae did not improve from 0.04437
Epoch 58/500
5/5 - 0s - loss: 0.0025 - mae: 0.0347 - val_loss: 0.0039 - val_mae: 0.0461

Epoch 00058: val_mae did not improve from 0.04437
Epoch 59/500
5/5 - 0s - loss: 0.0025 - mae: 0.0349 - val_loss: 0.0039 - val_mae: 0.0471

Epoch 00059: val_mae did not improve from 0.04437
Epoch 60/500
5/5 - 0s - loss: 0.0025 - mae: 0.0345 - val_loss: 0.0037 - val_mae: 0.0450

Epoch 00060: val_mae did not improve from 0.04437
Epoch 61/500
5/5 - 0s - loss: 0.0025 - mae: 0.0345 - val_loss: 0.0039 - val_mae: 0.0460

Epoch 00061: val_mae did not improve from 0.04437
Epoch 62/500
5/5 - 0s - loss: 0.0025 - mae: 0.0346 - val_loss: 0.0041 - val_mae: 0.0481

Epoch 00062: val_mae did not improve from 0.04437
Epoch 63/500
5/5 - 0s - loss: 0.0025 - mae: 0.0345 - val_loss: 0.0038 - val_mae: 0.0455

Epoch 00063: val_mae did n


Epoch 00114: val_mae did not improve from 0.04384
Epoch 115/500
5/5 - 0s - loss: 0.0024 - mae: 0.0341 - val_loss: 0.0038 - val_mae: 0.0458

Epoch 00115: val_mae did not improve from 0.04384
Epoch 116/500
5/5 - 0s - loss: 0.0024 - mae: 0.0338 - val_loss: 0.0038 - val_mae: 0.0457

Epoch 00116: val_mae did not improve from 0.04384
Epoch 117/500
5/5 - 0s - loss: 0.0024 - mae: 0.0338 - val_loss: 0.0036 - val_mae: 0.0439

Epoch 00117: val_mae did not improve from 0.04384
Epoch 118/500
5/5 - 0s - loss: 0.0025 - mae: 0.0347 - val_loss: 0.0038 - val_mae: 0.0460

Epoch 00118: val_mae did not improve from 0.04384
Epoch 119/500
5/5 - 0s - loss: 0.0024 - mae: 0.0341 - val_loss: 0.0038 - val_mae: 0.0456

Epoch 00119: val_mae did not improve from 0.04384
Epoch 120/500
5/5 - 0s - loss: 0.0024 - mae: 0.0338 - val_loss: 0.0037 - val_mae: 0.0451

Epoch 00120: val_mae did not improve from 0.04384
Epoch 121/500
5/5 - 0s - loss: 0.0024 - mae: 0.0337 - val_loss: 0.0036 - val_mae: 0.0440

Epoch 00121: val_ma


Epoch 00172: val_mae did not improve from 0.04382
Epoch 173/500
5/5 - 0s - loss: 0.0024 - mae: 0.0335 - val_loss: 0.0037 - val_mae: 0.0450

Epoch 00173: val_mae did not improve from 0.04382
Epoch 174/500
5/5 - 0s - loss: 0.0023 - mae: 0.0335 - val_loss: 0.0041 - val_mae: 0.0483

Epoch 00174: val_mae did not improve from 0.04382
Epoch 175/500
5/5 - 0s - loss: 0.0024 - mae: 0.0337 - val_loss: 0.0037 - val_mae: 0.0448

Epoch 00175: val_mae did not improve from 0.04382
Epoch 176/500
5/5 - 0s - loss: 0.0024 - mae: 0.0335 - val_loss: 0.0036 - val_mae: 0.0442

Epoch 00176: val_mae did not improve from 0.04382
Epoch 177/500
5/5 - 0s - loss: 0.0024 - mae: 0.0336 - val_loss: 0.0038 - val_mae: 0.0458

Epoch 00177: val_mae did not improve from 0.04382
Epoch 178/500
5/5 - 0s - loss: 0.0023 - mae: 0.0334 - val_loss: 0.0038 - val_mae: 0.0460

Epoch 00178: val_mae did not improve from 0.04382
Epoch 179/500
5/5 - 0s - loss: 0.0023 - mae: 0.0334 - val_loss: 0.0040 - val_mae: 0.0482

Epoch 00179: val_ma


Epoch 00231: val_mae did not improve from 0.04382
Epoch 232/500
5/5 - 0s - loss: 0.0023 - mae: 0.0332 - val_loss: 0.0036 - val_mae: 0.0445

Epoch 00232: val_mae did not improve from 0.04382
Epoch 233/500
5/5 - 0s - loss: 0.0023 - mae: 0.0333 - val_loss: 0.0038 - val_mae: 0.0462

Epoch 00233: val_mae did not improve from 0.04382
Epoch 234/500
5/5 - 0s - loss: 0.0023 - mae: 0.0331 - val_loss: 0.0036 - val_mae: 0.0443

Epoch 00234: val_mae did not improve from 0.04382
Epoch 235/500
5/5 - 0s - loss: 0.0023 - mae: 0.0333 - val_loss: 0.0037 - val_mae: 0.0447

Epoch 00235: val_mae did not improve from 0.04382
Epoch 236/500
5/5 - 0s - loss: 0.0023 - mae: 0.0333 - val_loss: 0.0038 - val_mae: 0.0455

Epoch 00236: val_mae did not improve from 0.04382
Epoch 237/500
5/5 - 0s - loss: 0.0023 - mae: 0.0332 - val_loss: 0.0038 - val_mae: 0.0465

Epoch 00237: val_mae did not improve from 0.04382
Epoch 238/500
5/5 - 0s - loss: 0.0023 - mae: 0.0330 - val_loss: 0.0036 - val_mae: 0.0448

Epoch 00238: val_ma


Epoch 00290: val_mae did not improve from 0.04382
Epoch 291/500
5/5 - 0s - loss: 0.0023 - mae: 0.0333 - val_loss: 0.0039 - val_mae: 0.0473

Epoch 00291: val_mae did not improve from 0.04382
Epoch 292/500
5/5 - 0s - loss: 0.0023 - mae: 0.0330 - val_loss: 0.0036 - val_mae: 0.0442

Epoch 00292: val_mae did not improve from 0.04382
Epoch 293/500
5/5 - 0s - loss: 0.0023 - mae: 0.0332 - val_loss: 0.0036 - val_mae: 0.0445

Epoch 00293: val_mae did not improve from 0.04382
Epoch 294/500
5/5 - 0s - loss: 0.0024 - mae: 0.0336 - val_loss: 0.0039 - val_mae: 0.0474

Epoch 00294: val_mae did not improve from 0.04382
Epoch 295/500
5/5 - 0s - loss: 0.0023 - mae: 0.0332 - val_loss: 0.0039 - val_mae: 0.0472

Epoch 00295: val_mae did not improve from 0.04382
Epoch 296/500
5/5 - 0s - loss: 0.0023 - mae: 0.0331 - val_loss: 0.0037 - val_mae: 0.0453

Epoch 00296: val_mae did not improve from 0.04382
Epoch 297/500
5/5 - 0s - loss: 0.0023 - mae: 0.0329 - val_loss: 0.0037 - val_mae: 0.0457

Epoch 00297: val_ma


Epoch 00348: val_mae did not improve from 0.04379
Epoch 349/500
5/5 - 0s - loss: 0.0023 - mae: 0.0329 - val_loss: 0.0037 - val_mae: 0.0458

Epoch 00349: val_mae did not improve from 0.04379
Epoch 350/500
5/5 - 0s - loss: 0.0023 - mae: 0.0328 - val_loss: 0.0037 - val_mae: 0.0456

Epoch 00350: val_mae did not improve from 0.04379
Epoch 351/500
5/5 - 0s - loss: 0.0023 - mae: 0.0331 - val_loss: 0.0037 - val_mae: 0.0452

Epoch 00351: val_mae did not improve from 0.04379
Epoch 352/500
5/5 - 0s - loss: 0.0023 - mae: 0.0335 - val_loss: 0.0039 - val_mae: 0.0477

Epoch 00352: val_mae did not improve from 0.04379
Epoch 353/500
5/5 - 0s - loss: 0.0024 - mae: 0.0337 - val_loss: 0.0041 - val_mae: 0.0488

Epoch 00353: val_mae did not improve from 0.04379
Epoch 354/500
5/5 - 0s - loss: 0.0023 - mae: 0.0335 - val_loss: 0.0037 - val_mae: 0.0451

Epoch 00354: val_mae did not improve from 0.04379
Epoch 355/500
5/5 - 0s - loss: 0.0023 - mae: 0.0329 - val_loss: 0.0038 - val_mae: 0.0459

Epoch 00355: val_ma


Epoch 00406: val_mae did not improve from 0.04364
Epoch 407/500
5/5 - 0s - loss: 0.0023 - mae: 0.0329 - val_loss: 0.0039 - val_mae: 0.0475

Epoch 00407: val_mae did not improve from 0.04364
Epoch 408/500
5/5 - 0s - loss: 0.0023 - mae: 0.0331 - val_loss: 0.0037 - val_mae: 0.0453

Epoch 00408: val_mae did not improve from 0.04364
Epoch 409/500
5/5 - 0s - loss: 0.0023 - mae: 0.0329 - val_loss: 0.0035 - val_mae: 0.0439

Epoch 00409: val_mae did not improve from 0.04364
Epoch 410/500
5/5 - 0s - loss: 0.0023 - mae: 0.0330 - val_loss: 0.0037 - val_mae: 0.0453

Epoch 00410: val_mae did not improve from 0.04364
Epoch 411/500
5/5 - 0s - loss: 0.0023 - mae: 0.0331 - val_loss: 0.0039 - val_mae: 0.0475

Epoch 00411: val_mae did not improve from 0.04364
Epoch 412/500
5/5 - 0s - loss: 0.0023 - mae: 0.0329 - val_loss: 0.0037 - val_mae: 0.0458

Epoch 00412: val_mae did not improve from 0.04364
Epoch 413/500
5/5 - 0s - loss: 0.0023 - mae: 0.0327 - val_loss: 0.0036 - val_mae: 0.0441

Epoch 00413: val_ma


Epoch 00465: val_mae did not improve from 0.04364
Epoch 466/500
5/5 - 0s - loss: 0.0023 - mae: 0.0328 - val_loss: 0.0038 - val_mae: 0.0466

Epoch 00466: val_mae did not improve from 0.04364
Epoch 467/500
5/5 - 0s - loss: 0.0022 - mae: 0.0326 - val_loss: 0.0038 - val_mae: 0.0470

Epoch 00467: val_mae did not improve from 0.04364
Epoch 468/500
5/5 - 0s - loss: 0.0022 - mae: 0.0326 - val_loss: 0.0037 - val_mae: 0.0456

Epoch 00468: val_mae did not improve from 0.04364
Epoch 469/500
5/5 - 0s - loss: 0.0023 - mae: 0.0328 - val_loss: 0.0036 - val_mae: 0.0445

Epoch 00469: val_mae did not improve from 0.04364
Epoch 470/500
5/5 - 0s - loss: 0.0023 - mae: 0.0330 - val_loss: 0.0038 - val_mae: 0.0467

Epoch 00470: val_mae did not improve from 0.04364
Epoch 471/500
5/5 - 0s - loss: 0.0022 - mae: 0.0326 - val_loss: 0.0037 - val_mae: 0.0453

Epoch 00471: val_mae did not improve from 0.04364
Epoch 472/500
5/5 - 0s - loss: 0.0023 - mae: 0.0326 - val_loss: 0.0036 - val_mae: 0.0445

Epoch 00472: val_ma

In [31]:
trained_cnn.compile(optimizer='adam',
                    loss='mse',
                    metrics=['mae']
                   )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint(
                      '../../saved_models/pressure_all_weighted_META_ft.h5', 
#                      '../../saved_models/pressure_all_weighted_EVT_95_ft.h5', 
#                      '../../saved_models/pressure_all_weighted_IPF_95_ft.h5', 
#                      '../../saved_models/pressure_all_95_ft.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True,
                    )


trained_cnn.fit(train_X_extreme, train_y_extreme,
                validation_data=(val_X_extreme, val_y_extreme),
                epochs=EPOCHS,
                batch_size=BATCH,
                verbose=2,
                shuffle=True,
                callbacks=[es, mc]
               )

Epoch 1/500
5/5 - 2s - loss: 0.0054 - mae: 0.0557 - val_loss: 0.0050 - val_mae: 0.0553

Epoch 00001: val_mae improved from inf to 0.05535, saving model to ../../saved_models/pressure_all_weighted_META_ft.h5
Epoch 2/500
5/5 - 0s - loss: 0.0037 - mae: 0.0449 - val_loss: 0.0049 - val_mae: 0.0523

Epoch 00002: val_mae improved from 0.05535 to 0.05226, saving model to ../../saved_models/pressure_all_weighted_META_ft.h5
Epoch 3/500
5/5 - 0s - loss: 0.0032 - mae: 0.0413 - val_loss: 0.0038 - val_mae: 0.0463

Epoch 00003: val_mae improved from 0.05226 to 0.04630, saving model to ../../saved_models/pressure_all_weighted_META_ft.h5
Epoch 4/500
5/5 - 0s - loss: 0.0030 - mae: 0.0389 - val_loss: 0.0048 - val_mae: 0.0540

Epoch 00004: val_mae did not improve from 0.04630
Epoch 5/500
5/5 - 0s - loss: 0.0029 - mae: 0.0378 - val_loss: 0.0037 - val_mae: 0.0448

Epoch 00005: val_mae improved from 0.04630 to 0.04479, saving model to ../../saved_models/pressure_all_weighted_META_ft.h5
Epoch 6/500
5/5 - 0s -

5/5 - 0s - loss: 0.0021 - mae: 0.0313 - val_loss: 0.0040 - val_mae: 0.0485

Epoch 00057: val_mae did not improve from 0.04331
Epoch 58/500
5/5 - 0s - loss: 0.0021 - mae: 0.0319 - val_loss: 0.0038 - val_mae: 0.0470

Epoch 00058: val_mae did not improve from 0.04331
Epoch 59/500
5/5 - 0s - loss: 0.0021 - mae: 0.0316 - val_loss: 0.0037 - val_mae: 0.0462

Epoch 00059: val_mae did not improve from 0.04331
Epoch 60/500
5/5 - 0s - loss: 0.0021 - mae: 0.0311 - val_loss: 0.0040 - val_mae: 0.0476

Epoch 00060: val_mae did not improve from 0.04331
Epoch 61/500
5/5 - 0s - loss: 0.0020 - mae: 0.0312 - val_loss: 0.0038 - val_mae: 0.0464

Epoch 00061: val_mae did not improve from 0.04331
Epoch 62/500
5/5 - 0s - loss: 0.0021 - mae: 0.0314 - val_loss: 0.0039 - val_mae: 0.0475

Epoch 00062: val_mae did not improve from 0.04331
Epoch 63/500
5/5 - 0s - loss: 0.0020 - mae: 0.0311 - val_loss: 0.0039 - val_mae: 0.0480

Epoch 00063: val_mae did not improve from 0.04331
Epoch 64/500
5/5 - 0s - loss: 0.0021 - m

5/5 - 0s - loss: 0.0016 - mae: 0.0284 - val_loss: 0.0056 - val_mae: 0.0586

Epoch 00116: val_mae did not improve from 0.04331
Epoch 117/500
5/5 - 0s - loss: 0.0017 - mae: 0.0288 - val_loss: 0.0056 - val_mae: 0.0584

Epoch 00117: val_mae did not improve from 0.04331
Epoch 118/500
5/5 - 0s - loss: 0.0017 - mae: 0.0288 - val_loss: 0.0054 - val_mae: 0.0573

Epoch 00118: val_mae did not improve from 0.04331
Epoch 119/500
5/5 - 0s - loss: 0.0017 - mae: 0.0287 - val_loss: 0.0055 - val_mae: 0.0583

Epoch 00119: val_mae did not improve from 0.04331
Epoch 120/500
5/5 - 0s - loss: 0.0016 - mae: 0.0283 - val_loss: 0.0052 - val_mae: 0.0571

Epoch 00120: val_mae did not improve from 0.04331
Epoch 121/500
5/5 - 0s - loss: 0.0017 - mae: 0.0285 - val_loss: 0.0054 - val_mae: 0.0574

Epoch 00121: val_mae did not improve from 0.04331
Epoch 122/500
5/5 - 0s - loss: 0.0016 - mae: 0.0275 - val_loss: 0.0059 - val_mae: 0.0594

Epoch 00122: val_mae did not improve from 0.04331
Epoch 123/500
5/5 - 0s - loss: 0.0


Epoch 00174: val_mae did not improve from 0.04331
Epoch 175/500
5/5 - 0s - loss: 0.0014 - mae: 0.0261 - val_loss: 0.0065 - val_mae: 0.0632

Epoch 00175: val_mae did not improve from 0.04331
Epoch 176/500
5/5 - 0s - loss: 0.0014 - mae: 0.0268 - val_loss: 0.0065 - val_mae: 0.0630

Epoch 00176: val_mae did not improve from 0.04331
Epoch 177/500
5/5 - 0s - loss: 0.0014 - mae: 0.0266 - val_loss: 0.0069 - val_mae: 0.0639

Epoch 00177: val_mae did not improve from 0.04331
Epoch 178/500
5/5 - 0s - loss: 0.0013 - mae: 0.0260 - val_loss: 0.0071 - val_mae: 0.0659

Epoch 00178: val_mae did not improve from 0.04331
Epoch 179/500
5/5 - 0s - loss: 0.0013 - mae: 0.0256 - val_loss: 0.0070 - val_mae: 0.0650

Epoch 00179: val_mae did not improve from 0.04331
Epoch 180/500
5/5 - 0s - loss: 0.0013 - mae: 0.0254 - val_loss: 0.0072 - val_mae: 0.0667

Epoch 00180: val_mae did not improve from 0.04331
Epoch 181/500
5/5 - 0s - loss: 0.0012 - mae: 0.0251 - val_loss: 0.0081 - val_mae: 0.0710

Epoch 00181: val_ma